In [81]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm

In [82]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
base_dir = '/content/drive/MyDrive/Skin desease detection with resnet34/Folder according to classes'

In [84]:
torch.manual_seed(42)

In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [86]:
train_transformations = transforms.Compose([
  transforms.Resize((256, 256)),
  transforms.RandomResizedCrop(224, scale=(0.8, 1)),
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomVerticalFlip(p=0.5),
  transforms.RandomRotation(30),
  transforms.ColorJitter(brightness=0.2),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      ])

val_test_transformations = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [87]:
dataset = ImageFolder(root=base_dir, transform=train_transformations)

In [88]:
len(dataset)

10015

In [89]:
train_size = int(0.45 * len(dataset))
val_size = int(0.20 * len(dataset))
test_size = len(dataset) - val_size - train_size

In [90]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [91]:
len(val_dataset)

2003

In [92]:
len(test_dataset)

3506

In [93]:
len(train_dataset)

4506

In [94]:
model = models.resnet34(pretrained = True)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [95]:
for name, params in model.named_parameters():
  if "fc" in name:
    params.requires_grad = False

In [96]:
model.fc.in_features

512

In [97]:
# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 1024),
#     nn.BatchNorm1d(1024),
#     nn.ReLU(),
#     nn.Dropout(0.5),

#     nn.Linear(1024, 512),
#     nn.BatchNorm1d(512),
#     nn.ReLU(),
#     nn.Dropout(0.5),

#     nn.Linear(512, 7)
# )

def create_model(dropout_rate):
  model.fc = nn.Sequential(
    nn.Linear(512, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(dropout_rate),

    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(dropout_rate),

    nn.Linear(512, 7)
  )
  return model

In [98]:
from collections import Counter

def weight_calculation(subset, num_classes):

  labels = [train_dataset.dataset.targets[i] for i in train_dataset.indices]
  class_counts = Counter(labels)

  weights = torch.zeros(num_classes)

  for class_idx, count in class_counts.items():
    weights[class_idx] = len(subset) / (num_classes * count)

  return weights

class_weights = weight_calculation(train_dataset, 7)
print(f"Class weights: {class_weights.round(decimals=3)}")

Class weights: tensor([ 4.3200,  2.8360,  1.2240, 10.9100,  1.2770,  0.2160, 10.2180])


In [99]:
# class_weights = class_weights.to(device)
# criterion = nn.CrossEntropyLoss(weight = class_weights)
# optimizer = optim.Adam(model.fc.parameters(), lr = 0.0001, weight_decay= 1e-5)

In [100]:
!pip install optuna

In [101]:
import optuna

def objective(trial):

  lr = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
  optimizer_name = trial.suggest_categorical("optimizer_name", ["adam", "adamw", "sgd"])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  model = create_model(dropout_rate = dropout_rate)
  model = model.to(device)

  if optimizer_name == "adam":
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay= weight_decay)
  elif optimizer_name == "adamw":
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay= weight_decay)
  if optimizer_name == "sgd":
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay= weight_decay, momentum = 0.9)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

  criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

  for epoch in range(3):
    model.train()
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    accuracy = correct / total
  return accuracy

In [102]:
## some things     i have to try later
# Missing scheduler - Important for convergence
# No early stopping - Could waste time on bad trials
# No differential learning rates - Important for transfer learning

In [ ]:
import optuna

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-08-05 09:16:25,749] A new study created in memory with name: no-name-4ff98152-2b0f-419c-9317-bde027c3b1e1
[I 2025-08-05 09:36:51,698] Trial 0 finished with value: 0.5336994508237644 and parameters: {'learning_rate': 0.0004110334381916631, 'dropout_rate': 0.4, 'batch_size': 64, 'optimizer_name': 'adamw', 'weight_decay': 0.0003300985827162464}. Best is trial 0 with value: 0.5336994508237644.
[I 2025-08-05 09:40:00,612] Trial 1 finished with value: 0.0654018971542686 and parameters: {'learning_rate': 0.0018271266767034841, 'dropout_rate': 0.5, 'batch_size': 16, 'optimizer_name': 'adam', 'weight_decay': 0.000596158668694593}. Best is trial 0 with value: 0.5336994508237644.
[I 2025-08-05 09:43:07,467] Trial 2 finished with value: 0.326510234648028 and parameters: {'learning_rate': 0.00863404578374377, 'dropout_rate': 0.30000000000000004, 'batch_size': 16, 'optimizer_name': 'adamw', 'weight_decay': 0.000432920666382502}. Best is trial 0 with value: 0.5336994508237644.
[I 2025-08-05 0

# first hyperparameter tuning run

In [ ]:
 # [I 2025-08-03 17:03:47,907] A new study created in memory with name: no-name-5d95d9ff-3b3c-4d27-a3a8-ddbe8ba74128
# [I 2025-08-03 18:01:16,151] Trial 0 finished with value: 0.6010983524712931 and parameters: {'learning_rate': 0.00013162905157524713, 'dropout_rate': 0.30000000000000004, 'batch_size': 8, 'optimizer_name': 'adam', 'weight_decay': 0.0007842340017548257}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:05:54,656] Trial 1 finished with value: 0.526709935097354 and parameters: {'learning_rate': 0.001058085700589979, 'dropout_rate': 0.2, 'batch_size': 16, 'optimizer_name': 'adamw', 'weight_decay': 5.314540572058576e-05}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:10:22,465] Trial 2 finished with value: 0.39191213180229656 and parameters: {'learning_rate': 1.3175960159530996e-05, 'dropout_rate': 0.30000000000000004, 'batch_size': 32, 'optimizer_name': 'sgd', 'weight_decay': 0.0005522645545297041}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:14:48,130] Trial 3 finished with value: 0.35197204193709436 and parameters: {'learning_rate': 0.007148659746153547, 'dropout_rate': 0.1, 'batch_size': 64, 'optimizer_name': 'adam', 'weight_decay': 0.0005318463132410022}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:19:14,256] Trial 4 finished with value: 0.29355966050923615 and parameters: {'learning_rate': 3.221568066879408e-05, 'dropout_rate': 0.5, 'batch_size': 64, 'optimizer_name': 'adam', 'weight_decay': 0.0003949652661861021}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:23:37,932] Trial 5 finished with value: 0.4213679480778832 and parameters: {'learning_rate': 0.0003060278815097925, 'dropout_rate': 0.5, 'batch_size': 32, 'optimizer_name': 'adam', 'weight_decay': 0.00040484913629545993}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:27:59,627] Trial 6 finished with value: 0.4028956565152272 and parameters: {'learning_rate': 0.00038992172401218267, 'dropout_rate': 0.4, 'batch_size': 32, 'optimizer_name': 'adamw', 'weight_decay': 0.0007731266012883096}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:32:21,813] Trial 7 finished with value: 0.09535696455317025 and parameters: {'learning_rate': 0.04985862344899001, 'dropout_rate': 0.1, 'batch_size': 32, 'optimizer_name': 'adam', 'weight_decay': 0.0002931392374893614}. Best is trial 0 with value: 0.6010983524712931.
# [I 2025-08-03 18:37:11,682] Trial 8 finished with value: 0.43534697953070395 and parameters: {'learning_rate': 0.001730153668904819, 'dropout_rate': 0.30000000000000004, 'batch_size': 8, 'optimizer_name': 'sgd', 'weight_decay': 0.0005897932708501489}. Best is trial 0 with value: 0.6010983524712931.

### second hyperparameter tuning run

In [ ]:
# ## second run

# [I 2025-08-05 09:16:25,749] A new study created in memory with name: no-name-4ff98152-2b0f-419c-9317-bde027c3b1e1
# [I 2025-08-05 09:36:51,698] Trial 0 finished with value: 0.5336994508237644 and parameters: {'learning_rate': 0.0004110334381916631, 'dropout_rate': 0.4, 'batch_size': 64, 'optimizer_name': 'adamw', 'weight_decay': 0.0003300985827162464}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:40:00,612] Trial 1 finished with value: 0.0654018971542686 and parameters: {'learning_rate': 0.0018271266767034841, 'dropout_rate': 0.5, 'batch_size': 16, 'optimizer_name': 'adam', 'weight_decay': 0.000596158668694593}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:43:07,467] Trial 2 finished with value: 0.326510234648028 and parameters: {'learning_rate': 0.00863404578374377, 'dropout_rate': 0.30000000000000004, 'batch_size': 16, 'optimizer_name': 'adamw', 'weight_decay': 0.000432920666382502}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:46:12,227] Trial 3 finished with value: 0.3659510733899151 and parameters: {'learning_rate': 0.0001412390038755788, 'dropout_rate': 0.2, 'batch_size': 32, 'optimizer_name': 'adam', 'weight_decay': 0.00037883974976798447}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:49:16,753] Trial 4 finished with value: 0.27309036445332 and parameters: {'learning_rate': 3.039179827011342e-05, 'dropout_rate': 0.4, 'batch_size': 32, 'optimizer_name': 'adamw', 'weight_decay': 4.8499515427154256e-05}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:52:27,578] Trial 5 finished with value: 0.36495257114328505 and parameters: {'learning_rate': 7.481862866230146e-05, 'dropout_rate': 0.4, 'batch_size': 16, 'optimizer_name': 'adam', 'weight_decay': 0.00028728775482701367}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:55:28,384] Trial 6 finished with value: 0.22266600099850226 and parameters: {'learning_rate': 0.020484924597062068, 'dropout_rate': 0.30000000000000004, 'batch_size': 64, 'optimizer_name': 'adam', 'weight_decay': 3.137362971838285e-05}. Best is trial 0 with value: 0.5336994508237644.
# [I 2025-08-05 09:58:31,009] Trial 7 finished with value: 0.5361957064403395 and parameters: {'learning_rate': 0.0009802727810440833, 'dropout_rate': 0.30000000000000004, 'batch_size': 64, 'optimizer_name': 'adam', 'weight_decay': 0.0001722744542523922}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:01:40,699] Trial 8 finished with value: 0.38242636045931105 and parameters: {'learning_rate': 7.593583061178741e-05, 'dropout_rate': 0.2, 'batch_size': 16, 'optimizer_name': 'adamw', 'weight_decay': 3.6366850050713694e-05}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:04:44,856] Trial 9 finished with value: 0.3939091362955567 and parameters: {'learning_rate': 0.00031109939187983887, 'dropout_rate': 0.2, 'batch_size': 32, 'optimizer_name': 'adam', 'weight_decay': 4.922156256967985e-05}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:07:43,351] Trial 10 finished with value: 0.5022466300549177 and parameters: {'learning_rate': 0.08730227186565202, 'dropout_rate': 0.1, 'batch_size': 64, 'optimizer_name': 'sgd', 'weight_decay': 9.606257505042079e-05}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:10:41,872] Trial 11 finished with value: 0.24812780828756864 and parameters: {'learning_rate': 0.0010607964968708994, 'dropout_rate': 0.4, 'batch_size': 64, 'optimizer_name': 'sgd', 'weight_decay': 0.00016659291596290561}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:13:42,600] Trial 12 finished with value: 0.33399900149775336 and parameters: {'learning_rate': 0.0028473081391680102, 'dropout_rate': 0.5, 'batch_size': 64, 'optimizer_name': 'adamw', 'weight_decay': 1.1044765696386432e-05}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:16:40,113] Trial 13 finished with value: 0.3904143784323515 and parameters: {'learning_rate': 0.0005202387274291231, 'dropout_rate': 0.4, 'batch_size': 64, 'optimizer_name': 'adamw', 'weight_decay': 0.0008573286857763861}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:19:40,229] Trial 14 finished with value: 0.38092860708936593 and parameters: {'learning_rate': 0.00554449868227217, 'dropout_rate': 0.30000000000000004, 'batch_size': 64, 'optimizer_name': 'sgd', 'weight_decay': 0.000159779361446123}. Best is trial 7 with value: 0.5361957064403395.
# [I 2025-08-05 10:22:38,033] Trial 15 finished with value: 0.36844732900649024 and parameters: {'learning_rate': 1.0724194711558139e-05, 'dropout_rate': 0.5, 'batch_size': 64, 'optimizer_name': 'adam', 'weight_decay': 0.00019552336094963348}. Best is trial 7 with value: 0.5361957064403395.
